In [ ]:
from pathlib import Path
import pandas as pd
import re

log_paths = [Path(d, 'checkpoint.log') for d in Path('/Users/marwei/AFS/home/urz/m/marwei/Logs/CoLLAs2023/where_to_split_resnet_10mib_cifar10').glob('cifar10_*')] \
          + [Path(d, 'checkpoint.log') for d in Path('/Users/marwei/AFS/home/urz/m/marwei/Logs/CoLLAs2023/where_to_split_resnet_10mib_cifar100').glob('cifar100_*')]

dd = []
for exp in log_paths:
    with open(exp) as infile:
        loglines = infile.read().splitlines()
    dd.append(
        {
            'memory_size': int(re.findall(r"memory_size=(\d+)", loglines[0])[0]),
            'accuracy': float(re.findall(r"Acc: \[(.*?)\]", loglines[-1])[0]),
            'dataset': re.findall(r"dataset=\'(.*?)\'", loglines[0])[0],
            'backbone_block': int(re.findall(r"backbone_block=(\d+)", loglines[0])[0]),
            'seed': int(re.findall(r"seed=(\d+)", loglines[0])[0]),
        }
    )
df = pd.DataFrame.from_records(dd).sort_values('memory_size')

In [ ]:
df2 = df.groupby(['dataset', 'backbone_block', 'memory_size'])['accuracy'].agg(['median', 'min', 'max']).reset_index()
df2['error_pos'] = df2['max'] - df2['median'] 
df2['error_neg'] = df2['median'] - df2['min']
df2.sort_values('memory_size', inplace=True)

In [ ]:
import plotly.express as px
from plot_utils import science_template

fig = px.line(
    df2,
    x='memory_size',
    y='median',
    text='backbone_block',
    color='dataset',
    template=science_template,
    labels={
        'dataset': '',
        'median': 'Accuracy',
        'memory_size': 'N'
    },
    range_y=[0, 0.8]
)

fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.25,
        xanchor="left",
        x=0,
        font_size=16
    ),
    font={
        'size': 16,
    },
    yaxis_title_standoff=3,
    xaxis_title_standoff=15
)

fig.write_image('../plots/wts.pdf', width=500, height=420)
fig.show()